This code is to download all the lenses at the coordinates described by a csv file which you provide the path to. The coordinates might be in the form

- 333.094995,-0.303149,Jacobs
- 00:49:41.89,-27:52:25.7,CASTLES
- 09 59 29.0,+01 45 53,faure

In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:90% !important;}</style>'))


In [ ]:
#get cutouts of lenses from papers

import pandas as pd
import subprocess
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from daomop.storage import tap_query
from astropy import units as u
from astropy.coordinates import SkyCoord
CSV = '/home/toyonagar/Documents/paper_crds.csv'
CUTOUT_PATH ="/home/toyonagar/anaconda3/HST_Lens/paper_cutouts/"

def get_filenames(ra, dec,i):
    if (":" in str(ra)) or (" " in str(ra)):
        try:
            crd = SkyCoord(ra[0:2]+"h"+ra[3:5]+'m'+ra[6:10]+'s',dec[0:3]+"d"+dec[4:6]+'m'+dec[7:9]+'s')
            ra = crd.ra.degree
            dec = crd.dec.degree

        except:
            print('unable to convert crds to degree')
            print(ra)
            return 0
        
    query = '''SELECT caom2.Plane.publisherID, caom2.Plane.planeURI 
    FROM caom2.observation 
    JOIN caom2.plane ON caom2.observation.obsID = caom2.plane.obsID 
    WHERE CONTAINS( CIRCLE('ICRS',''' + str(ra) + ',' +str(dec) + ''',0.001), caom2.plane.position_bounds)=1 
    AND caom2.observation.collection = 'HST' 
    AND caom2.observation.instrument_name = 'ACS/WFC' 
    AND caom2.Plane.energy_bandpassName ='F814W'
    ''' #'F814W' F606W,F850W F475W
    try:
        out_query = tap_query(query)
    except: 
        print('tap query failed')
        print(ra,dec)
        return 0
    
    '''store the pubID and filenames'''
    success=False
    for entry in range(len(out_query["publisherID"])):
        if ("0-" in out_query["publisherID"][entry]) and ("PRODUCT" in out_query["publisherID"][entry]):
            success=True
            start = out_query["publisherID"][entry].find('HST?') + 4 #add 4 to get to the and of the HST?
            end = out_query["publisherID"][entry].find('/j', start)
            print(out_query["publisherID"][entry][start:end])
            try:
                subprocess.check_output(["wget", "-O", CUTOUT_PATH+papers['author'][i]+"_"+str(i)+'.fits',
                                 "http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/caom2ops/cutout?uri=mast:HST/product/"+
                                 str(out_query["publisherID"][entry][start:end])+"_drz.fits&cutout=Circle+ICRS+" + str(ra) + "+" +
                                 str(dec) + "+0.002"])
                break
            except:
                print('failed download')
    if success==False: print('couldnt find filename '+str(i))




papers = pd.read_csv(CSV, sep = ',')

for i in range(len(papers)):
    #for each row try to download the image
#     try:
        r = get_filenames(papers['ra'][i], papers['dec'][i],i)
#     except:
#         print((papers['ra'][i], papers['dec'][i],i)) 

In [ ]:
### load images from files


import os 
from astropy.io import fits 
import numpy as np 
from sklearn.preprocessing import minmax_scale
from astropy.nddata.utils import Cutout2D

PATH ="/home/toyonagar/anaconda3/HST_Lens/faure_100x100_no-norm/"
ims = np.zeros((len(os.listdir(PATH)),1,100,100))
# sp= int(input("start point"))

count=0
for filename in os.listdir(PATH): 
    try:
        pic = fits.open(PATH+filename)[0].data
        
        if pic.shape != (100, 100):    #check for cut off images
            result = np.zeros((100,100))
            result[:pic.shape[0],:pic.shape[1]] = pic
            ims[count][0] = result
            count+=1
            continue
            
        ims[count][0] = pic
        count+=1
         
    except:
        print('missed'+ str(i+sp))


